In [265]:
from simtk.openmm.app import *
from openforcefield.utils import *

In [266]:
help (get_data_file_path)

Help on function get_data_file_path in module openforcefield.utils.utils:

get_data_file_path(relative_path)
    Get the full path to one of the reference files in testsystems.
    In the source distribution, these files are in ``openforcefield/data/``,
    but on installation, they're moved to somewhere in the user's python
    site-packages directory.
    Parameters
    ----------
    name : str
        Name of the file to load (with respect to the repex folder).



In [267]:
pdb_file_path = get_data_file_path ('Acetylacetone.pdb')
pdbfile = PDBFile (pdb_file_path)

# I got the pdb file from
# https://www.ebi.ac.uk/pdbe-srv/pdbechem/chemicalCompound/show/P2D
# renamed it, then I manually moved it to the data directory of openforcefield.

In [268]:
from openforcefield.topology import Molecule
acetylacetone = Molecule.from_smiles('O=C(C)CC(=O)C')
print(acetylacetone)

# I got the SMILES-code from:
# https://en.wikipedia.org/wiki/Acetylacetone

Molecule with name '' and SMILES '[H][C]([H])([H])[C](=[O])[C]([H])([H])[C](=[O])[C]([H])([H])[H]'


In [269]:
print (pdb_file_path)

/home/jules/anaconda3/lib/python3.7/site-packages/openforcefield/data/Acetylacetone.pdb


In [270]:
from openforcefield.topology import *
from openforcefield.typing.engines.smirnoff import ForceField

omm_topology =  pdbfile.topology
off_topology = Topology.from_openmm(omm_topology, unique_molecules=[acetylacetone])

forcefield = ForceField('openff-1.0.0.offxml')

system = forcefield.create_openmm_system(off_topology)


In [271]:
from simtk import *

# Langevin Dynamics:
time_step = 2*unit.femtoseconds
temperature = 300*unit.kelvin
friction = 1/unit.picosecond # collision rate
integrator = openmm.LangevinIntegrator(temperature, friction, time_step)

# Lenghth of simulation:
num_steps = 1

trj_freq = 1 # number of steps per written trajectory frame
data_freq = 1 #number of steps per fritten simulation statistics

# Set up OpenMM simulation:
simulation = openmm.app.Simulation (omm_topology, system, integrator)

# initial positions:
positions = pdbfile.getPositions()
simulation.context.setPositions(positions)

# Randomize the velocities from a Boltzmann distribution at a given temperature.
simulation.context.setVelocitiesToTemperature(temperature)

# MINIMIZE ENERGY: I deleted this line to get the non-minimized values:
simulation.minimizeEnergy()
minimized_coords = simulation.context.getState(getPositions=True).getPositions()
minimized_forces = simulation.context.getState(getForces=True).getForces()

# Configure information in output files:
pdb_reporter = openmm.app.PDBReporter('trajectory.pdb', trj_freq)
state_data_reporter = openmm.app.StateDataReporter ('data.csv', data_freq, step=True, potentialEnergy=True, temperature=True, density=True)
simulation.reporters.append (pdb_reporter)
simulation.reporters.append (state_data_reporter)

In [272]:
import time

print("Starting simulation")
start = time.process_time()

# Run the simulation
simulation.step(num_steps)

end = time.process_time()
print("Elapsed time %.2f seconds" % (end-start))
print("Hurra!")

Starting simulation
Elapsed time 0.00 seconds
Hurra!


In [273]:
ff_applied_parameters = forcefield.label_molecules(off_topology)[0]
ff_values=[]
ff_valuefile = open ('ff_valuefile.txt', 'w+')

for atoms, bonds in ff_applied_parameters['Bonds'].items():
    ff_valuefile.write (F'{atoms},{bonds}')
    ff_valuefile.write ('\n')
    
ff_valuefile.close()

In [274]:
import numpy

ff_valuefile = open ('ff_valuefile.txt', 'r')

ff_valuetab = numpy.genfromtxt(fname=ff_valuefile, dtype='unicode', delimiter='Lenght')

ff_valuefile.close

man_atom_a = [0,1,1,2,2,2,3,3,3,4,4,6,6,6]
man_atom_b = [1,2,3,7,8,9,4,10,11,5,6,12,13,14]

In [275]:
import numpy
import os

mincoorfile = os.path.join('trajectory.pdb')

mincoor = numpy.genfromtxt(fname=mincoorfile, skip_header=2, skip_footer=2, dtype='unicode')

print (mincoor)

[['HETATM' '1' 'C1' 'P2D' 'A' '1' '2.270' '-0.167' '0.034' '1.00' '0.00'
  'C']
 ['HETATM' '2' 'C2' 'P2D' 'A' '1' '1.029' '-0.816' '-0.557' '1.00' '0.00'
  'C']
 ['HETATM' '3' 'C3' 'P2D' 'A' '1' '0.000' '-1.451' '0.295' '1.00' '0.00'
  'C']
 ['HETATM' '4' 'C4' 'P2D' 'A' '1' '-1.032' '-0.394' '0.604' '1.00' '0.00'
  'C']
 ['HETATM' '5' 'C5' 'P2D' 'A' '1' '-2.265' '-0.241' '-0.269' '1.00'
  '0.00' 'C']
 ['HETATM' '6' 'O2' 'P2D' 'A' '1' '0.857' '-0.794' '-1.774' '1.00' '0.00'
  'O']
 ['HETATM' '7' 'O4' 'P2D' 'A' '1' '-0.818' '0.399' '1.502' '1.00' '0.00'
  'O']
 ['HETATM' '8' 'H1' 'P2D' 'A' '1' '2.791' '-0.886' '0.671' '1.00' '0.00'
  'H']
 ['HETATM' '9' 'H2' 'P2D' 'A' '1' '2.887' '0.204' '-0.789' '1.00' '0.00'
  'H']
 ['HETATM' '10' 'H3' 'P2D' 'A' '1' '1.959' '0.655' '0.683' '1.00' '0.00'
  'H']
 ['HETATM' '11' 'H4' 'P2D' 'A' '1' '-0.437' '-2.312' '-0.217' '1.00'
  '0.00' 'H']
 ['HETATM' '12' 'H5' 'P2D' 'A' '1' '0.437' '-1.807' '1.232' '1.00' '0.00'
  'H']
 ['HETATM' '13' 'H6' 'P2D' 'A' 

In [276]:
xyz_data = mincoor[:,6:9]
xyz_data = xyz_data.astype(numpy.float)
print (xyz_data)

xa = xyz_data[man_atom_a,0]
xb = xyz_data[man_atom_b,0]
ya = xyz_data[man_atom_a,1]
yb = xyz_data[man_atom_b,1]
za = xyz_data[man_atom_a,2]
zb = xyz_data[man_atom_b,2]

[[ 2.27  -0.167  0.034]
 [ 1.029 -0.816 -0.557]
 [ 0.    -1.451  0.295]
 [-1.032 -0.394  0.604]
 [-2.265 -0.241 -0.269]
 [ 0.857 -0.794 -1.774]
 [-0.818  0.399  1.502]
 [ 2.791 -0.886  0.671]
 [ 2.887  0.204 -0.789]
 [ 1.959  0.655  0.683]
 [-0.437 -2.312 -0.217]
 [ 0.437 -1.807  1.232]
 [-2.891 -1.136 -0.249]
 [-2.836  0.607  0.118]
 [-1.975 -0.023 -1.3  ]]


In [277]:
bondlength_min=numpy.sqrt(((xa-xb)**2)+((ya-yb)**2)+((za-zb)**2))

print (bondlength_min)    

[1.52005362 1.47917883 2.40285788 2.87233041 3.49983571 2.90231993
 1.51849498 2.16951377 2.13281832 3.50966067 2.37483684 3.11762009
 2.4558184  3.06070858]
